In [ ]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
ROOT_DIR = "/content/drive/MyDrive/Colab Notebooks/"
RES_DIR = ROOT_DIR + "resources/"
MOD_DIR = ROOT_DIR + "ml_models/"
BATCH_SIZE = 32

X = pd.read_csv(RES_DIR + "X_NEG0.csv")["Sysnet"]
y = pd.read_csv(RES_DIR + "y_NEG0.csv")["NEG"]


In [ ]:
BUFFER_SIZE = 1000
BATCH_SIZE = 128


In [ ]:
X_pom, X_val, y_pom, y_val = train_test_split(X, y, test_size=.2, stratify=y, )
X_train, X_test, y_train, y_test = train_test_split(X_pom, y_pom, test_size=.1, stratify=y_pom, )

X_val = tf.convert_to_tensor(X_val, name ="Definicija")
y_val = tf.convert_to_tensor(y_val, name ="Sentiment")


X_train = tf.convert_to_tensor(X_train, name ="Definicija")
y_train = tf.convert_to_tensor(y_train, name ="Sentiment")

X_test = tf.convert_to_tensor(X_test, name ="Definicija")
y_test = tf.convert_to_tensor(y_test, name ="Sentiment")

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
validation_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))


In [ ]:
tf.math.confusion_matrix(labels=y_train, predictions=y_train).numpy()


array([[14127,     0],
       [    0,   157]], dtype=int32)

In [ ]:
train_dataset =train_dataset.map(lambda x_text, x_label: (x_text, tf.expand_dims(x_label, -1)))
validation_dataset= validation_dataset.map(lambda x_text, x_label: (x_text, tf.expand_dims(x_label, -1)))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [ ]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
validation_dataset = validation_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
tf.shape(X_train)

<tf.Tensor: shape=(1,), dtype=int32, numpy=array([14284], dtype=int32)>

In [ ]:
import tensorflow_datasets as tfds
import numpy as np


In [1]:
VOCAB_SIZE = 25000
encoder = tf.keras.layers.TextVectorization(standardize=None,
    max_tokens=VOCAB_SIZE, output_mode="int" )
encoder.adapt(train_dataset.map(lambda text, label: text))



NameError: ignored

In [ ]:
vocab = np.array(encoder.get_vocabulary())
vocab[:30]


array(['', '[UNK]', 'koji', 'se', 'i', 'u', 'ili', 'na', 'jesam', 'od',
       'za', 'sa', 'iz', 'neki', 'da', 'imati', 'rod', 'koristiti', 'kao',
       'nešto', 'biljka', 'velik', 'jezik', 'biti', 'card', 'porodica',
       'jedan', 'mali', 'drugi', 'deo'], dtype='<U21')

In [ ]:
len(encoder.get_vocabulary())

16752

In [ ]:
model_NEG = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(len(encoder.get_vocabulary()), 128, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

In [ ]:
model_NEG.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['binary_accuracy'])

In [ ]:
history = model_NEG.fit(train_dataset, epochs=10,
                    validation_data=validation_dataset,
                    validation_steps=30)

Epoch 1/10
112/112 [==============================] - 152s 1s/step - loss: 0.4931 - binary_accuracy: 0.9586 - val_loss: 0.1470 - val_binary_accuracy: 0.9885
Epoch 2/10
112/112 [==============================] - 102s 913ms/step - loss: 0.1465 - binary_accuracy: 0.9837 - val_loss: 0.1085 - val_binary_accuracy: 0.9885
Epoch 3/10
112/112 [==============================] - 106s 943ms/step - loss: 0.1177 - binary_accuracy: 0.9863 - val_loss: 0.0996 - val_binary_accuracy: 0.9885
Epoch 4/10
112/112 [==============================] - 109s 979ms/step - loss: 0.1091 - binary_accuracy: 0.9876 - val_loss: 0.0911 - val_binary_accuracy: 0.9885
Epoch 5/10
112/112 [==============================] - 101s 906ms/step - loss: 0.0965 - binary_accuracy: 0.9880 - val_loss: 0.0872 - val_binary_accuracy: 0.9885
Epoch 6/10
112/112 [==============================] - 107s 961ms/step - loss: 0.0856 - binary_accuracy: 0.9888 - val_loss: 0.0848 - val_binary_accuracy: 0.9885
Epoch 7/10
112/112 [=======================

In [ ]:
model_NEG.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 128)         2144256   
                                                                 
 bidirectional (Bidirectiona  (None, None, 256)        263168    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, None, 128)        164352    
 nal)                                                            
                                                                 
 bidirectional_2 (Bidirectio  (None, None, 64)         41216     
 nal)                                                   

In [ ]:
y_pred = model_NEG.predict(X_test)

50/50 [==============================] - 16s 76ms/step


In [ ]:
# model.save(MOD_DIR + "nntest3")

In [ ]:
y_pred[:10]

array([[0.00016713],
       [0.00015823],
       [0.00014666],
       [0.00017026],
       [0.00035831],
       [0.00027721],
       [0.00017633],
       [0.00024541],
       [0.00023078],
       [0.00020245]], dtype=float32)

In [ ]:
y_pred = tf.round((y_pred))

In [ ]:
con_mat = tf.math.confusion_matrix(labels=y_test, predictions=y_pred).numpy()


In [ ]:
con_mat

array([[1563,    7],
       [  13,    5]], dtype=int32)

In [ ]:
y_pred = model_NEG.predict(X_val)

125/125 [==============================] - 12s 93ms/step


In [ ]:
con_mat = tf.math.confusion_matrix(labels=y_val, predictions=y_pred).numpy()


In [ ]:
con_mat

array([[3925,    0],
       [  44,    0]], dtype=int32)

In [ ]:
X = pd.read_csv(RES_DIR + "X_POS0.csv")["Sysnet"]
y = pd.read_csv(RES_DIR + "y_POS0.csv")["POS"]

In [ ]:
X_pom, X_val, y_pom, y_val = train_test_split(X, y, test_size=.1, stratify=y, )
X_train, X_test, y_train, y_test = train_test_split(X_pom, y_pom, test_size=.2, stratify=y_pom, )

X_val = tf.convert_to_tensor(X_val, name ="Definicija")
y_val = tf.convert_to_tensor(y_val, name ="Sentiment")


X_train = tf.convert_to_tensor(X_train, name ="Definicija")
y_train = tf.convert_to_tensor(y_train, name ="Sentiment")

X_test = tf.convert_to_tensor(X_test, name ="Definicija")
y_test = tf.convert_to_tensor(y_test, name ="Sentiment")

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
validation_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))


In [ ]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
validation_dataset = validation_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE, output_mode="int" )
encoder.adapt(train_dataset.map(lambda text, label: text))



In [ ]:
model_POS = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(len(encoder.get_vocabulary()), 128, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

In [ ]:
model_POS.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['binary_accuracy'])

In [ ]:
history = model_POS.fit(train_dataset, epochs=10,
                    validation_data=validation_dataset,
                    validation_steps=30)

Epoch 1/10
112/112 [==============================] - ETA: 0s - loss: 0.4102 - binary_accuracy: 0.9882

112/112 [==============================] - 151s 1s/step - loss: 0.4102 - binary_accuracy: 0.9882 - val_loss: 0.1094 - val_binary_accuracy: 0.9924
Epoch 2/10
112/112 [==============================] - 91s 816ms/step - loss: 0.1054 - binary_accuracy: 0.9924
Epoch 3/10
112/112 [==============================] - 97s 866ms/step - loss: 0.0855 - binary_accuracy: 0.9925
Epoch 4/10
112/112 [==============================] - 95s 839ms/step - loss: 0.0731 - binary_accuracy: 0.9922
Epoch 5/10
112/112 [==============================] - 98s 879ms/step - loss: 0.0672 - binary_accuracy: 0.9926
Epoch 6/10
112/112 [==============================] - 97s 863ms/step - loss: 0.0616 - binary_accuracy: 0.9926
Epoch 7/10
112/112 [==============================] - 96s 857ms/step - loss: 0.0488 - binary_accuracy: 0.9930
Epoch 8/10
112/112 [==============================] - 98s 874ms/step - loss: 0.0358 - binary_accuracy: 0.9947
Epoch 9/10
112/112 [==============================] - 94s 835ms/step - loss: 0.0269 

In [ ]:
model_POS.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_1 (TextV  (None, None)             0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, None, 128)         2142848   
                                                                 
 bidirectional_4 (Bidirectio  (None, None, 256)        263168    
 nal)                                                            
                                                                 
 bidirectional_5 (Bidirectio  (None, None, 128)        164352    
 nal)                                                            
                                                                 
 bidirectional_6 (Bidirectio  (None, None, 64)         41216     
 nal)                                                 

In [ ]:
y_pred = model_POS.predict(X_test)

112/112 [==============================] - 20s 87ms/step


In [ ]:
# model.save(MOD_DIR + "nntest3")

In [ ]:
y_pred[:10]

array([[1.3385122e-05],
       [1.9817258e-05],
       [1.0375911e-03],
       [1.3717387e-05],
       [1.2708180e-04],
       [1.8061310e-04],
       [1.1310963e-04],
       [2.2235026e-05],
       [8.3913423e-05],
       [1.0320977e-03]], dtype=float32)

In [ ]:
y_pred = tf.round((y_pred))

In [ ]:
con_mat = tf.math.confusion_matrix(labels=y_test, predictions=y_pred).numpy()


In [ ]:
con_mat

array([[3539,    7],
       [  16,   10]], dtype=int32)

In [ ]:
y_pred = model_POS.predict(X_val)

63/63 [==============================] - 5s 78ms/step


In [ ]:
con_mat = tf.math.confusion_matrix(labels=y_val, predictions=y_pred).numpy()


In [ ]:
con_mat

array([[1970,    0],
       [  15,    0]], dtype=int32)

In [ ]:
def polarity_correction(pos, neg):
  one = tf.convert_to_tensor(1.0)
  ret_pos = pos*(one - neg)
  ret_neg =neg * (one -pos)
  return ret_pos, ret_neg
  

In [ ]:
sword = pd.read_csv(RES_DIR +"definicije_lematizone.csv", index_col=0)

In [ ]:
definicije = sword["Definicija"]

In [ ]:
tensor_def = tf.convert_to_tensor(definicije.to_numpy(dtype='str'))

In [ ]:
tn_POS = model_POS.predict(tensor_def)
tn_NEG = model_NEG.predict(tensor_def)


792/792 [==============================] - 59s 75ms/step


In [ ]:
tn_POSc, tn_NEGc = polarity_correction(tn_POS, tn_NEG)

In [ ]:
sword["POS"] = tn_POSc.numpy()

In [ ]:
sword["NEG"] = tn_NEGc.numpy()

In [ ]:
sword

,ID,Definicija,POS,NEG
0,ENG30-03574555-n,zgrada u koji se nalaziti organizacioni jedini...,0.000020,0.000158
1,ENG30-07810907-n,pripremljen dodatak jela za poboljšanje ukus,0.000119,0.000213
2,ENG30-00721431-n,"u nečiji prilika , mogućnost",0.000221,0.000669
3,ENG30-00473799-v,ostati jesam još samo da se doterati neki fine...,0.000025,0.000167
4,ENG30-00903385-v,zapad on jesam oprostiti što se nekada računat...,0.000121,0.000182
...,...,...,...,...
25315,ENG30-15266265-n,mandat predsednik,0.068185,0.080432
25316,ENG30-15266685-n,vreme između početak i kraj vremenski perioda,0.000068,0.000181
25317,ENG30-15266911-n,tačka u vreme kada se nešto završavati,0.000076,0.000178
25318,ENG30-15276642-n,( računarstvo ) brzina prenos podatak ( npr . ...,0.000081,0.000189


In [ ]:
sword.to_csv(RES_DIR + "sentiment_RNN0.csv")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')